In [1]:
from stravaio import StravaIO
from stravaio import strava_oauth2
import requests
import json

with open('strava_ids.json','r') as f:
    strava_data = json.loads(f.read())

strava_auth_data = strava_oauth2(strava_data['client_id'], strava_data['client_secret'])
strava_client = StravaIO(strava_auth_data['access_token'])    

2021-06-28 18:50:49.435 | INFO     | stravaio:strava_oauth2:343 - serving at port 8000
2021-06-28 18:50:52.085 | DEBUG    | stravaio:run_server_and_wait_for_token:397 - code: 41eca40dae670be920cf175e6e441de0deeb2733
2021-06-28 18:50:52.699 | DEBUG    | stravaio:run_server_and_wait_for_token:406 - Authorized athlete: 82d5dcc98eac153519d5ef5df8abd62a38c8e2b1


In [2]:
activity_id = 5518106799
headers = {"Authorization": "Bearer {access_token}".format(access_token = strava_auth_data['access_token'])}
activity_url = 'https://www.strava.com/api/v3/activities/{activity_id}?include_all_efforts'.format(activity_id = activity_id)
response = requests.get(activity_url,headers=headers)
activity_dict = json.loads(response.text)

#distance data stream
url = 'https://www.strava.com/api/v3/activities/{activity_id}/streams?'.format(activity_id = activity_id)
response = requests.get(url,headers=headers)
#print(response)
#print(response.text)
distance_dict = json.loads(response.text[1:len(response.text)-1])
distance_data = distance_dict['data']

#time data stream
url = 'https://www.strava.com/api/v3/activities/{activity_id}/streams?&type=time&series_type=time'.format(activity_id = activity_id)
response = requests.get(url,headers=headers)
#print(response)
time_dict = json.loads(response.text[1:len(response.text)-1])
time_data = time_dict['data']

In [3]:
import pandas as pd
from datetime import datetime, timedelta
df = pd.DataFrame({'time':time_data, 'distance': distance_data})
df['time_diff'] = df['time'].shift(-1) - df['time']
df['dist_diff'] = df['distance'].shift(-1) - df['distance']

start_time_str = activity_dict['start_date_local']
start_time = datetime.strptime(start_time_str, '%Y-%m-%dT%H:%M:%SZ')

total_dist = delta_dist = delta_time = i = 0
timestamps, pace ,distance = ([],[],[])
elapsed_time = start_time

for row in df.iterrows():
    delta_dist += row[1][3]
    delta_time += row[1][2]
    if delta_dist >= 100:
        elapsed_time += timedelta(seconds=delta_time)
        timestamps.append(elapsed_time)
        pace.append(delta_time/60)
        distance.append(total_dist + delta_dist)
        i += 1
        delta_dist = delta_time = 0
       
print(i)  

29


In [4]:
from oauth_server import OAuth2Server
import fitbit
with open('fitbit_ids.json','r') as f:
    fitbit_data = json.loads(f.read())
fitbit_oauthserver = OAuth2Server(fitbit_data['client_id'], fitbit_data['client_secret'])
fitbit_oauthserver.browser_authorize()

[28/Jun/2021:18:51:07] ENGINE Listening for SIGTERM.
[28/Jun/2021:18:51:07] ENGINE Listening for SIGHUP.
[28/Jun/2021:18:51:07] ENGINE Listening for SIGUSR1.
[28/Jun/2021:18:51:07] ENGINE Bus STARTING
CherryPy Checker:
The Application mounted at '' has an empty config.

[28/Jun/2021:18:51:07] ENGINE Started monitor thread 'Autoreloader'.
[28/Jun/2021:18:51:07] ENGINE Serving on http://127.0.0.1:8080
[28/Jun/2021:18:51:07] ENGINE Bus STARTED


127.0.0.1 - - [28/Jun/2021:18:51:09] "GET /?code=a5352a3893c1d4d331e64aa97d06e5afeb604c0a&state=BBLqfAvwOhhkfaKDZmCFi5vblW6E5d HTTP/1.1" 200 122 "" "Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/89.0.4389.90 Safari/537.36"


[28/Jun/2021:18:51:10] ENGINE Bus STOPPING
[28/Jun/2021:18:51:10] ENGINE HTTP Server cherrypy._cpwsgi_server.CPWSGIServer(('127.0.0.1', 8080)) shut down
[28/Jun/2021:18:51:10] ENGINE Stopped thread 'Autoreloader'.
[28/Jun/2021:18:51:10] ENGINE Bus STOPPED
[28/Jun/2021:18:51:10] ENGINE Bus EXITING
[28/Jun/2021:18:51:10] ENGINE Bus EXITED
[28/Jun/2021:18:51:10] ENGINE Waiting for child threads to terminate...


In [5]:
client = fitbit.Fitbit(fitbit_data['client_id'], fitbit_data['client_secret'],
         access_token = fitbit_oauthserver.fitbit.client.session.token['access_token'],
         refresh_token = fitbit_oauthserver.fitbit.client.session.token['refresh_token'])

heart_url =  "https://api.fitbit.com/1.2/user/-/activities/heart/date/{date}/1d/1sec.json".format(date = start_time.strftime("%Y-%m-%d"))
fitbit_hr_data = client.make_request(heart_url)
heart_data = fitbit_hr_data['activities-heart-intraday']
hr_dataset = heart_data['dataset']
hr_times, hb_values = ([],[])

end_time = start_time + timedelta(seconds = activity_dict['elapsed_time'])    
for h in hr_dataset:
    date_time_obj = datetime.strptime(h['time'], '%H:%M:%S')
    date_time_obj = date_time_obj.replace(year=start_time.year, month=start_time.month, day = start_time.day)
    if date_time_obj > start_time and date_time_obj < end_time:
        hr_times.append(date_time_obj)
        hb_values.append(h['value'])
       

In [6]:
from bokeh.plotting import figure, output_file, show
from bokeh.models import LinearAxis, Range1d
from bokeh.models import HoverTool

from math import pi
chart_red = '#f45666'
chart_blue = '#4292c6'

chart_tools="tap, crosshair, wheel_zoom,zoom_in,zoom_out"

p = figure(title="swimming pace vs heart beat", 
           x_axis_type="datetime", x_axis_label='time', 
           y_axis_label = 'pace [min/100m]', y_range=(2, 8),
           toolbar_location="below",tools=chart_tools)
p.yaxis.axis_line_color = chart_blue
p.yaxis.major_tick_line_color = chart_blue
p.yaxis.minor_tick_line_color = chart_blue

p.add_tools(HoverTool(
    tooltips=[('time', '$x{%H:%M:%S}'),
              ("value", "$y{0.00}")],
    formatters={'$x':"datetime"}
))

#pace
p.line(timestamps,pace,  color = chart_blue)

#heartbeat
p.extra_y_ranges = {"heart_beat": Range1d(start=60, end=150),}

# axis_label_text_color or major_label_text_color
p.add_layout(LinearAxis(y_range_name="heart_beat", 
                        axis_label="hear beat [bpm]", 
                        axis_line_color = chart_red,
                        major_tick_line_color = chart_red,
                        minor_tick_line_color = chart_red),
                        'right')
p.scatter(hr_times, hb_values, color = chart_red, y_range_name="heart_beat")


output_file("pace_vs_heartbit.html", title="swimming pace vs heartbeat")
show(p)  